# TorchMD API 教程

## 系统设置

我们使用 `moleculekit` 库来读取输入拓扑和初始坐标

In [ ]:
from moleculekit.molecule import Molecule
import os

testdir = "../tests/data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # 读取系统拓扑
mol.read(os.path.join(testdir, "input.coor"))  # 读取初始模拟坐标
mol.read(os.path.join(testdir, "input.xsc"))  # 读取盒子尺寸

接下来我们将加载一个力场文件，并使用上面的拓扑来提取相关参数，这些参数将用于模拟

In [2]:
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters
import torch

precision = torch.float
device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))
parameters = Parameters(ff, mol, precision=precision, device=device)

现在我们可以创建一个 `System` 对象，它将包含模拟期间系统的状态，包括：
1. 当前原子坐标
1. 当前盒子大小
1. 当前原子速度
1. 当前原子力

In [3]:
from torchmd.integrator import maxwell_boltzmann
from torchmd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

最后我们将创建一个 `Force` 对象，它将用于在给定的 `System` 状态上评估势能

In [4]:
from torchmd.forces import Forces

forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5, terms=["bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj"])
# 评估当前能量和力，力会就地修改
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'bonds': 3.957749366760254, 'angles': 2.844572067260742, 'dihedrals': 10.579870223999023, 'impropers': 1.2417081594467163, '1-4': 0.0, 'electrostatics': -2568.497802734375, 'lj': 359.25079345703125, 'external': 0.0}]
tensor([[[  3.0404,   1.7028,   3.8141],
         [-15.2398, -17.4599,   5.3314],
         [  2.5749,   3.8611,  -4.1888],
         ...,
         [-22.4462,   8.8784,  32.4494],
         [  1.1741,  -8.0141, -15.6699],
         [ 20.2039,  -3.2618, -10.9875]]], device='cuda:0')


## 分子动力学

为了执行动力学，我们将创建一个 `Integrator` 对象来积分模拟的时间步长，以及一个 `Wrapper` 对象来将系统坐标包装在周期性胞元内

In [5]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300  # K 开尔文
langevin_gamma = 0.1
timestep = 1  # fs 飞秒

integrator = Integrator(system, forces, timestep, device, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None, device)

In [6]:
from torchmd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # 最小化系统

Iter  Epot            fmax    
   0   -2190.623291    64.694467
   1   -1917.852417    153.854104
   2   -2307.939453    37.247713
   3   -2360.717529    24.187750
   4   -2389.794434    18.115517
   5   -2406.956299    13.061655
   6   -2453.368652    17.315425
   7   -2481.085693    87.200834
   8   -2511.368896    30.966888
   9   -2522.753174    26.002184
  10   -2530.704346    12.667158
  11   -2540.423096    23.035339
  12   -2553.544678    37.369995
  13   -2570.214355    33.325471
  14   -2555.467285    92.054068
  15   -2578.091309    38.819339
  16   -2589.268555    15.860706
  17   -2594.457031    16.899775
  18   -2600.699707    19.500992
  19   -2609.083740    32.366387
  20   -2617.950439    17.982933
  21   -2622.973389    16.235269
  22   -2629.241699    15.279906
  23   -2632.602783    20.167234
  24   -2638.844238    19.689059
  25   -2645.946533    41.638018
  26   -2652.022461    27.241360
  27   -2654.648926    19.046010
  28   -2658.422119    9.623597
  29   -2663

为模拟创建一个CSV文件记录器，用于跟踪能量和温度。

In [ ]:
from torchmd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')
if hasattr(logger, 'f') and logger.f:
    logger.f.close()

Writing logs to  logs/monitor.csv


现在我们最终可以执行完整的分子动力学模拟

In [10]:
from tqdm import tqdm 
import numpy as np
import os

# 确保logs目录存在
os.makedirs("logs", exist_ok=True)

FS2NS = 1E-6 # 飞秒到纳秒的转换

steps = 1000
output_period = 10
save_period = 100
traj = []

# 所有输出文件保存到logs目录
trajectoryout = "logs/mytrajectory.npy"

iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    # 移除replica维度，只保存原子坐标
    currpos = currpos[0] if currpos.ndim == 3 else currpos  # 形状: (natoms, 3)
    traj.append(currpos)
    
    if (i*output_period) % save_period  == 0:
        # 保存为 (natoms, 3, nframes) 格式
        traj_array = np.stack(traj, axis=2)
        np.save(trajectoryout, traj_array)

# 最后保存完整轨迹
if traj:
    traj_array = np.stack(traj, axis=2)
    np.save(trajectoryout, traj_array)
    print(f"轨迹形状: {traj_array.shape}")  # 应该是 (natoms, 3, nframes)

# 转换为VMD兼容的XYZ格式
from torchmd.utils import xyz_writer

# 获取原子元素信息 (这里是丙氨酸二肽的元素序列)
elements = []
for i in range(mol.numAtoms):
    atom_name = mol.name[i]
    if atom_name.startswith('H'):
        elements.append('H')
    elif atom_name.startswith('C'):
        elements.append('C')
    elif atom_name.startswith('N'):
        elements.append('N')
    elif atom_name.startswith('O'):
        elements.append('O')
    else:
        elements.append('C')  # 默认为碳

# 转换轨迹为XYZ格式供VMD使用，保存到logs目录
xyz_output = "logs/trajectory_for_vmd.xyz"

# 检查轨迹文件是否存在并转换
if os.path.exists(trajectoryout):
    xyz_writer(trajectoryout, xyz_output, elements)
    
    print(f"所有输出文件已保存到logs目录:")
    print(f"- 轨迹文件: {trajectoryout}")
    print(f"- VMD格式: {xyz_output}")
    print(f"- 监控数据: logs/monitor.csv")
    print(f"\n在VMD中使用:")
    print(f"mol new ../tests/data/prod_alanine_dipeptide_amber/structure.pdb")
    print(f"mol addfile {xyz_output}")
else:
    print("轨迹文件未生成，请检查模拟是否完成")

100%|██████████| 100/100 [00:22<00:00,  4.50it/s]

轨迹形状: (688, 3, 100)
所有输出文件已保存到logs目录:
- 轨迹文件: logs/mytrajectory.npy
- VMD格式: logs/trajectory_for_vmd.xyz
- 监控数据: logs/monitor.csv

在VMD中使用:
mol new ../tests/data/prod_alanine_dipeptide_amber/structure.pdb
mol addfile logs/trajectory_for_vmd.xyz
